In [2]:
from doc2vec import *
import sys

## Initialization
In `server.py`, we use these lines of codes for initialization:
```python
i = sys.argv.index('server:app')
glove_filename = sys.argv[i+1]
articles_dirname = sys.argv[i+2]

gloves = load_glove(glove_filename)
articles_loaded = load_articles(articles_dirname, gloves)   
```
How do them work?
First, when we are launching the Web server using gunicorn: 
```
gunicorn -D --threads 4 -b 0.0.0.0:5000 --access-logfile server.log --timeout 60 server:app glove.6B.300d.txt bbc
```
We need to find the glove and articles arguments. The first three lines of codes realize it. It will read `glove.6B.300d.txt` into `glove_filename` variable and then `bbc` into `articles_dirname` variable.

In [28]:
glove_filename = 'glove.6B.300d.txt'
articles_dirname = 'bbc'

The `gloves` variable is the dictionary mapping a word to its 300-vector vector.

In [5]:
gloves = load_glove(glove_filename)

In [18]:
gloves['the'].shape, gloves['dog'].shape

((300,), (300,))

The `articles` is a list of records, one for each article. An article record is just a list containing the fully-qualified file name, the article title, the text without the title, and the word vector computed from the text without the title.

In [12]:
articles_loaded = load_articles(articles_dirname, gloves)   

In [16]:
print('number of acticle record: ', len(articles_loaded))

number of acticle record:  2226


In [23]:
article = articles_loaded[0]
article

('entertainment/289.txt',
 'Musicians to tackle US red tape',
 '\nMusicians\' groups are to tackle US visa regulations which are blamed for hindering British acts\' chances of succeeding across the Atlantic.\n\nA singer hoping to perform in the US can expect to pay $1,300 (Â£680) simply for obtaining a visa. Groups including the Musicians\' Union are calling for an end to the "raw deal" faced by British performers. US acts are not faced with comparable expense and bureaucracy when visiting the UK for promotional purposes.\n\nNigel McCune from the Musicians\' Union said British musicians are "disadvantaged" compared to their US counterparts. A sponsor has to make a petition on their behalf, which is a form amounting to nearly 30 pages, while musicians face tougher regulations than athletes and journalists. "If you make a mistake on your form, you risk a five-year ban and thus the ability to further your career," says Mr McCune.\n\n"The US is the world\'s biggest music market, which mean

## Recommendation
After we read `gloves` (database of word vectors) and `articles_loaded ` (a corpus of text article records) into memory, we can use word2vec to recommend articles.
To get the list of most relevant five articles for `article`, we'll do this:

```python
similar_articles = recommended(article, articles_loaded, 5)
```

In [29]:
similar_articles = recommended(article, articles_loaded, 5)

`similar_articles` is a list containing 5 article records.

### How does `recommended` work?
What is process when we run `recommended(article, articles_loaded, 5)`?

First, it will run `distances(article, articles)`, which is going to compute the euclidean distance from `article` to every other article and return a list of (distance, a) tuples for all a in articles.

In [30]:
distances_list = distances(article, articles_loaded)

In [43]:
print(f'For example, the euclidean distance from article to the last article in the articles_loaded is {distances_list[-1][0]}.')

For example, the euclidean distance from article to the last article in the articles_loaded is 2.1802242628836463.


Then, it will sort `distances_list` in reverse order by distance.

In [44]:
sorted_distances_list = sorted(distances_list, key = lambda x:x[0], reverse=False) 

Finally, it will return the top n (n is 5 in this example) article records. 

In [48]:
similar_articles = [record[1] for record in sorted_distances_list[:5]]